In [34]:
import numpy as np
import pprint
import copy

# Helper Funcs

In [53]:
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)

In [1]:
def initialize_parameters(n_a, n_x, n_y):
    np.random.seed(1)
    
    Wax = np.random.randn(n_a, n_x) * 0.01 
    Waa = np.random.randn(n_a, n_a) * 0.01
    Wya = np.random.randn(n_y, n_a) * 0.01
    ba = np.zeros((n_a, 1)) # hidden bias
    by = np.zeros((n_y, 1)) # output bias
    
    parameters = {"Wax": Wax, "Waa": Waa, "Wya": Wya, "ba": ba,"by": by}
    return parameters

In [5]:
def rnn_step_forward(parameters, a_prev, x):
    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    
    a_next = np.tanh(np.dot(Wax, x) + np.dot(Waa, a_prev) + ba)
    p_t = softmax(np.dot(Wya, a_next) + by)
    
    return a_next, p_t

In [ ]:
def rnn_step_backward(dy, gradients, parameters, x, a, a_prev):
    
    gradients['dWya'] += np.dot(dy, a.T)
    gradients['dby'] += dy
    da = np.dot(parameters['Wya'].T, dy) + gradients['da_next'] # backprop into h
    daraw = (1 - a * a) * da # backprop through tanh nonlinearity
    gradients['db'] += daraw
    gradients['dWax'] += np.dot(daraw, x.T)
    gradients['dWaa'] += np.dot(daraw, a_prev.T)
    gradients['da_next'] = np.dot(parameters['Waa'].T, daraw)
    return gradients

In [21]:
data = open('dinos.txt', mode='r').read().lower()
chars = list(set(data))

data_size, vocab_size = len(data), len(chars)
print('There are %d total characters and %d unique characters in your data.' % (data_size, vocab_size))

There are 19909 total characters and 27 unique characters in your data.


In [25]:
chars = sorted(chars)
print(chars)

['\n', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


* The characters are a-z (26 characters) plus the "\n" (or newline character).
* In this assignment, the newline character "\n" plays a role similar to the `<EOS>` (or "End of sentence") token discussed in lecture.  
    - Here, "\n" indicates the end of the dinosaur name rather than the end of a sentence. 

In [48]:
char_to_idx = {ch:i for i,ch in enumerate(chars)}
idx_to_char = {i:ch for i,ch in enumerate(chars)}

print(char_to_idx)

{'\n': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


* `char_to_idx`: In the cell below, you'll create a Python dictionary (i.e., a hash table) to map each character to an index from 0-26.
* `idx_to_char`: Then, you'll create a second Python dictionary that maps each index back to the corresponding character. 
    -  This will help you figure out which index corresponds to which character in the probability distribution output of the softmax layer. 

In [36]:
def clip_gradients(gradients, max_value):
    '''
    Clips the gradients' values between minimum and maximum.
    
    Arguments:
    gradients -- a dictionary containing the gradients "dWaa", "dWax", "dWya", "dba", "dby"
    max_value -- everything above this number is set to this number, and everything less than -max_value is set to -max_value
    
    Returns: 
    gradients -- a dictionary with the clipped gradients.
    '''
    
    gradients = copy.deepcopy(gradients)
    
    dWaa, dWax, dWya, dba, dby = gradients['dWaa'], gradients['dWax'], gradients['dWya'], gradients['dba'], gradients['dby']
    for grad in [dWaa, dWax, dWya, dba, dby]:
        np.clip(grad, -max_value, max_value, out=grad)
    
    gradients = {"dWaa": dWaa, "dWax": dWax, "dWya": dWya, "dba": dba, "dby": dby}
    return gradients

In [ ]:
def sample(parameters, char_to_idx, seed):
    """
    Sample a sequence of characters according to a sequence of probability distributions output of the RNN

    Arguments:
    parameters -- Python dictionary containing the parameters Waa, Wax, Wya, ba, and by. 
    char_to_idx -- Python dictionary mapping each character to an index.
    seed -- Used for grading purposes. Do not worry about it.

    Returns:
    indices -- A list of length n containing the indices of the sampled characters.
    """

    Waa, Wax, Wya, by, ba = parameters['Waa'], parameters['Wax'], parameters['Wya'], parameters['by'], parameters['ba']
    vocab_size = by.shape[0]  # by (n_y, 1)
    n_a = Waa.shape[1]

    x = np.zeros((vocab_size, 1))
    a_prev = np.zeros((n_a, 1))

    idx = -1
    counter = 0
    newline_char = char_to_idx['\n']
    while (idx != newline_char and counter != 50):
        a = np.tanh(np.dot(Waa, a_prev) + np.dot(Wax, x) + ba)
        z = np.dot(Wya, a) + by
        y = softmax(z)  # y (n_y, m)

        idx = np.random.choice(list(range(vocab_size)), p = y.ravel())  
        # Append the index to "indices"
        indices.append(idx)

In [73]:
idx = np.random.choice(list(range(vocab_size)), p = y.ravel())

13